In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.functions import udf
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number
#from pyspark.sql.functions import lead
#from pyspark.sql.functions import min, max
#from pyspark.sql.functions import unix_timestamp

In [3]:
sc = SparkContext(appName='Nyc Death Causes')

22/08/21 20:40:11 WARN Utils: Your hostname, rodrigo-Dell-G15-5511 resolves to a loopback address: 127.0.1.1; using 192.168.15.132 instead (on interface wlp0s20f3)
22/08/21 20:40:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/21 20:40:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc

<SparkContext master=local[*] appName=Nyc Death Causes>

In [5]:
spark = SparkSession.builder.getOrCreate()

In [12]:
file = '../data/New_York_City_Leading_Causes_of_Death.csv'
nycDF = spark.read.csv(file,inferSchema=True, header=True)

In [13]:
nycDF.show(5)

+----+--------------------+---+--------------------+------+----------+-----------------------+
|Year|       Leading Cause|Sex|      Race Ethnicity|Deaths|Death Rate|Age Adjusted Death Rate|
+----+--------------------+---+--------------------+------+----------+-----------------------+
|2009|Nephritis, Nephro...|  F|Other Race/ Ethni...|     .|         .|                      .|
|2013|Influenza (Flu) a...|  F|            Hispanic|   204|      16.3|                   18.5|
|2012|Assault (Homicide...|  M|Other Race/ Ethni...|     .|         .|                      .|
|2007|Essential Hyperte...|  F|  Not Stated/Unknown|     5|         .|                      .|
|2014|Cerebrovascular D...|  F|  White Non-Hispanic|   418|      29.5|                   15.6|
+----+--------------------+---+--------------------+------+----------+-----------------------+
only showing top 5 rows



In [24]:
nycDF = nycDF.replace(['Female','Male'],['F','M'])

In [25]:
nycDF.groupBy("Sex").count().show()

+---+-----+
|Sex|count|
+---+-----+
|  F|  644|
|  M|  628|
+---+-----+



In [48]:
nycDF.agg({"Deaths":'mean'}).collect()

[Row(avg(Deaths)=422.88977072310405)]

In [47]:
nycDF.groupBy('Year').agg({'Deaths':'sum'}).sort("Year").show()

+----+-----------+
|Year|sum(Deaths)|
+----+-----------+
|2007|    53996.0|
|2008|    54138.0|
|2009|    52820.0|
|2010|    52505.0|
|2011|    52726.0|
|2012|    52420.0|
|2013|    53387.0|
|2014|    53006.0|
|2019|    54559.0|
+----+-----------+



In [34]:
nycDF.groupBy('Race Ethnicity').count().show()

+--------------------+-----+
|      Race Ethnicity|count|
+--------------------+-----+
|  Black Non-Hispanic|  178|
|  Not Stated/Unknown|  223|
|  White Non-Hispanic|  176|
|  Non-Hispanic Black|   22|
|Asian and Pacific...|  199|
|Other Race/ Ethni...|  253|
|            Hispanic|  199|
|  Non-Hispanic White|   22|
+--------------------+-----+



In [37]:
nycDF.groupBy('Leading Cause').count().sort('count', ascending = False).show()

+--------------------+-----+
|       Leading Cause|count|
+--------------------+-----+
|    All Other Causes|  110|
|Influenza (Flu) a...|  110|
|Diseases of Heart...|  110|
|Malignant Neoplas...|  110|
|Diabetes Mellitus...|  106|
|Cerebrovascular D...|  103|
|Chronic Lower Res...|  100|
|Essential Hyperte...|   88|
|Accidents Except ...|   80|
|Mental and Behavi...|   51|
|Human Immunodefic...|   44|
|Alzheimer's Disea...|   40|
|Intentional Self-...|   38|
|Chronic Liver Dis...|   29|
|Certain Condition...|   28|
|Assault (Homicide...|   20|
|Nephritis, Nephro...|   19|
|Septicemia (A40-A41)|   16|
|Accidents Except ...|   13|
|Congenital Malfor...|   10|
+--------------------+-----+
only showing top 20 rows



In [45]:
nycDF.where(nycDF.Sex == 'M').groupBy("Leading Cause").count().sort('count', ascending = False).show()

+--------------------+-----+
|       Leading Cause|count|
+--------------------+-----+
|    All Other Causes|   55|
|Influenza (Flu) a...|   55|
|Diseases of Heart...|   55|
|Malignant Neoplas...|   55|
|Diabetes Mellitus...|   52|
|Chronic Lower Res...|   49|
|Cerebrovascular D...|   48|
|Accidents Except ...|   45|
|Essential Hyperte...|   35|
|Mental and Behavi...|   33|
|Human Immunodefic...|   27|
|Intentional Self-...|   26|
|Chronic Liver Dis...|   21|
|Assault (Homicide...|   17|
|Certain Condition...|   14|
|Accidents Except ...|    7|
|Mental and Behavi...|    5|
|Intentional Self-...|    4|
|Viral Hepatitis (...|    4|
|Nephritis, Nephro...|    3|
+--------------------+-----+
only showing top 20 rows



In [46]:
nycDF.where(nycDF.Sex == 'F').groupBy("Leading Cause").count().sort('count', ascending = False).show()

+--------------------+-----+
|       Leading Cause|count|
+--------------------+-----+
|Cerebrovascular D...|   55|
|Diseases of Heart...|   55|
|    All Other Causes|   55|
|Malignant Neoplas...|   55|
|Influenza (Flu) a...|   55|
|Diabetes Mellitus...|   54|
|Essential Hyperte...|   53|
|Chronic Lower Res...|   51|
|Alzheimer's Disea...|   37|
|Accidents Except ...|   35|
|Mental and Behavi...|   18|
|Human Immunodefic...|   17|
|Nephritis, Nephro...|   16|
|Certain Condition...|   14|
|Septicemia (A40-A41)|   13|
|Intentional Self-...|   12|
|Chronic Liver Dis...|    8|
|Congenital Malfor...|    7|
|Insitu or Benign ...|    6|
|Accidents Except ...|    6|
+--------------------+-----+
only showing top 20 rows

